In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!unzip '/content/gdrive/MyDrive/enron11.zip' -d '/content'

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/enron1/ham/0241.2000-01-21.farmer.ham.txt  
  inflating: /content/enron1/ham/0242.2000-01-21.farmer.ham.txt  
  inflating: /content/enron1/ham/0243.2000-01-24.farmer.ham.txt  
  inflating: /content/enron1/ham/0244.2000-01-24.farmer.ham.txt  
  inflating: /content/enron1/ham/0245.2000-01-24.farmer.ham.txt  
  inflating: /content/enron1/ham/0246.2000-01-24.farmer.ham.txt  
  inflating: /content/enron1/ham/0247.2000-01-24.farmer.ham.txt  
  inflating: /content/enron1/ham/0249.2000-01-24.farmer.ham.txt  
  inflating: /content/enron1/ham/0251.2000-01-24.farmer.ham.txt  
  inflating: /content/enron1/ham/0252.2000-01-25.farmer.ham.txt  
  inflating: /content/enron1/ham/0253.2000-01-25.farmer.ham.txt  
  inflating: /content/enron1/ham/0254.2000-01-25.farmer.ham.txt  
  inflating: /content/enron1/ham/0255.2000-01-25.farmer.ham.txt  
  inflating: /content/enron1/ham/0257.2000-01-25.farmer.ham.txt  
  inflating: /content/enron1/ham/0258.2000-01-25.

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Aug 24 00:33:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
print("============查看cpu配置================")
# 查看cpu配置
!cat /proc/cpuinfo | grep model\ name
print("=============查看内存容量===============")
# 查看内存容量
!cat /proc/meminfo | grep MemTotal

============查看cpu配置================
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
=============查看内存容量===============
MemTotal:       13298580 kB


In [5]:
#四层加dense层(第一轮胜利)
#调整unit 128 最终胜出
#


import random
import tensorflow as tf
seed = 1

import random
import numpy as np
import tensorflow as tf
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


# importing system libraries

from os import walk
from string import punctuation
from random import shuffle
from collections import Counter
from sklearn.model_selection import cross_val_predict
# importing additional libraries

import pandas as pd
import sklearn as sk
import nltk
import pytest
nltk.download('stopwords')
import time
start_time = time.time()



from nltk.test.classify_fixt import setup_module
setup_module()
from os import walk
from string import punctuation
from random import shuffle
from collections import Counter
from sklearn.model_selection import cross_val_predict
# importing additional libraries

import pandas as pd
import sklearn as sk
import nltk
import pytest




from nltk.test.classify_fixt import setup_module
setup_module()

import pandas as pd
import numpy as np
import re
import collections

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import keras
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle





# Read the whole data from the Enron Dataset into a variable allData.

pathwalk = walk(r"/content/enron1/")

allHamData, allSpamData = [], []
for root, dr, file in pathwalk:
    if 'ham' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                allHamData.append(" ".join(ip.readlines()))
                
    elif 'spam' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                allSpamData.append(" ".join(ip.readlines()))
                
                
                

allHamData = list(set(allHamData))
allSpamData = list(set(allSpamData))


hamPlusSpamData = allHamData + allSpamData
labels = ["ham"]*len(allHamData) + ["spam"]*len(allSpamData)

raw_df = pd.DataFrame({"email": hamPlusSpamData, 
                       "label": labels})



raw_df.sample(5)
raw_df['email']

""""""""""""""""""""""""""""""""""""""""""


def preprocess(data):
    # tokenization
    tokens = nltk.word_tokenize(data)
    tokens = [w.lower() for w in tokens if w.isalpha()]

    # finding uncommon words
    cnt = Counter(tokens)
    uncommons = cnt.most_common()[:-int(len(cnt)*0.1):-1]
    
    # listing stopwords from NLTK
    stops = set(nltk.corpus.stopwords.words('english'))

    # removing stop words and uncommon words
    tokens = [w for w in tokens if (w not in stops and w not in uncommons)]

    # lemmatization
    lemmatizer = nltk.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w, pos='a') for w in tokens]

    return tokens

label_encoder = sk.preprocessing.LabelEncoder()
raw_df['label'] = label_encoder.fit_transform(raw_df.label)
import string
from nltk.corpus import stopwords

def process_text(text):
    no_punc = [char for char in text if char not in string.punctuation]
    no_punc = ''.join(no_punc)
    return ' '.join([word for word in no_punc.split() if word.lower() not in stopwords.words('english')])

raw_df['email']=raw_df['email'].apply(process_text)
raw_df['email']

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stemming (text):
    return ''.join([stemmer.stem(word) for word in text])
raw_df['email']=raw_df['email'].apply(stemming)
raw_df.head()


from sklearn.feature_extraction.text import CountVectorizer
vectorizer= CountVectorizer()
message_bow = vectorizer.fit_transform(raw_df['email'])

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(message_bow,raw_df['label'],test_size=0.25)

X = message_bow
y = raw_df['label']




# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(message_bow)

# Transform entire BoW into tf-idf corpus
message_bow_tfidf = tfidf_transformer.transform(message_bow)

X_tfidf = message_bow_tfidf
y = raw_df['label']
X_tfidf_train,X_tfidf_test,y_tfidf_train,y_tfidf_test = train_test_split(message_bow_tfidf,raw_df['label'],test_size=0.25)



texts1 = np.asarray(message_bow)
labels1 = np.asarray(raw_df['label'])



import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

import keras
from keras.models import Sequential
from keras.layers import Dense


from keras.layers import SimpleRNN, Embedding, Dense, LSTM
from keras.models import Sequential

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# number of words used as features
max_features = 500000
# cut off the words after seeing 500 words in each document(email)
maxlen = 500


# we will use 80% of data as training, 20% as validation data
training_samples = int(4994 * .75)
validation_samples = int(4994 - training_samples)
# sanity check
print(len(raw_df['email']) == (training_samples + validation_samples))
print("The number of training {0}, validation {1} ".format(training_samples, validation_samples))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(raw_df['email'])
sequences = tokenizer.texts_to_sequences(raw_df['email'])

word_index = tokenizer.word_index
print("Found {0} unique words: ".format(len(word_index)))

data = pad_sequences(sequences, maxlen=maxlen)

print("data shape: ", data.shape)


#os.environ['PYTHONHASHSEED']=str(SEED)
#random.seed(SEED) 
#np.random.seed(SEED)
#tf.random.set_seed(SEED)
#tf.compat.v1.set_random_seed(123)
# shuffle data
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = np.asarray(labels)
labels = y[indices]


texts_train = data[:training_samples]
y_train = labels[:training_samples]
texts_test = data[training_samples:]
y_test = labels[training_samples:]


from tensorflow.keras.optimizers import RMSprop


max_features =500000
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128))
model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=RMSprop(learning_rate=0.0005), loss='binary_crossentropy', metrics=['acc'])
history_ltsm = model.fit(texts_train, y_train, epochs=15, batch_size=16, shuffle=False)

pred = (model.predict(texts_test) > 0.5).astype("int32")

acc = model.evaluate(texts_test, y_test)
proba_ltsm = (model.predict(texts_test) > 0.5).astype("int32")
from sklearn.metrics import confusion_matrix
print("Test loss is {0:.4f} accuracy is {1:.4f}  ".format(acc[0],acc[1]))

from sklearn.metrics import classification_report
print("LSTM score",classification_report(y_test,pred, digits=4))
print(confusion_matrix(pred, y_test))

from pandas.core.common import random_state

from keras.layers.core import Flatten



def create_keras_model():

    max_features =500000
    model2 = Sequential()
    model2.add(Embedding(max_features, 128))
    model2.add(LSTM(128, return_sequences=True))
    model2.add(LSTM(128, return_sequences=True))
    model2.add(LSTM(128, return_sequences=True))
    model2.add(LSTM(128))
    model2.add(Dense(128, activation='relu'))
    model2.add(Dense(128, activation='relu'))
    model2.add(Dense(1, activation='sigmoid'))
#    rmsprop1 = RMSprop(learning_rate = 0.1)
    model2.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['acc'])



    return model2


from keras.wrappers.scikit_learn import KerasClassifier

LSTMmodel1 = KerasClassifier(
        build_fn=create_keras_model, 
        batch_size=32, epochs=50)


LSTMmodel1._estimator_type = "classifier"






from sklearn.model_selection import cross_val_score
#accuracies_LSTM1 = cross_val_score(estimator=LSTMmodel1, scoring="accuracy", X=data, y=labels, cv=10)
#accuracies_LSTM2 = cross_val_score(estimator=LSTMmodel1, scoring="precision", X=data, y=labels, cv=10)
#print("LSTM corss validation :  %0.4f accuracy with a standard deviation of %0.4f" % (accuracies_LSTM1.mean(), accuracies_LSTM1.std()))
#print("LSTM corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_LSTM2.mean(), accuracies_LSTM2.std()))


accuracies_LSTMmodel1_accuracy = cross_val_score(estimator=LSTMmodel1, scoring="accuracy", 
    X=data, y=labels, cv=5)
accuracies_LSTMmodel1_f1 = cross_val_score(estimator=LSTMmodel1, scoring="f1", 
    X=data, y=labels, cv=5)
accuracies_LSTMmodel1_recall = cross_val_score(estimator=LSTMmodel1, scoring="recall", 
    X=data, y=labels, cv=5)
accuracies_LSTMmodel1_precision = cross_val_score(estimator=LSTMmodel1, scoring="precision", 
    X=data, y=labels, cv=5)

print("unit 128")
print("----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("LSTMmodel1 corss validation :  %0.4f accuracy  with a standard deviation of %0.4f" % (accuracies_LSTMmodel1_accuracy.mean(), accuracies_LSTMmodel1_accuracy.std()))
print("LSTMmodel1 corss validation :  %0.4f f1     with a standard deviation of %0.4f" % (accuracies_LSTMmodel1_f1.mean(), accuracies_LSTMmodel1_f1.std()))
print("LSTMmodel1 corss validation :  %0.4f recall   with a standard deviation of %0.4f" % (accuracies_LSTMmodel1_recall.mean(), accuracies_LSTMmodel1_recall.std()))
print("LSTMmodel1 corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_LSTMmodel1_precision.mean(), accuracies_LSTMmodel1_precision.std()))






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True
The number of training 3745, validation 1249 
Found 50422 unique words: 
data shape:  (4994, 500)
Epoch 1/15
235/235 [==============================] - 14s 30ms/step - loss: 0.2245 - acc: 0.9087
Epoch 2/15
235/235 [==============================] - 7s 29ms/step - loss: 0.0427 - acc: 0.9874
Epoch 3/15
235/235 [==============================] - 7s 29ms/step - loss: 0.0201 - acc: 0.9949
Epoch 4/15
235/235 [==============================] - 7s 29ms/step - loss: 0.0172 - acc: 0.9968
Epoch 5/15
235/235 [==============================] - 7s 29ms/step - loss: 0.0044 - acc: 0.9989
Epoch 6/15
235/235 [==============================] - 7s 29ms/step - loss: 0.0109 - acc: 0.9992
Epoch 7/15
235/235 [==============================] - 7s 29ms/step - loss: 0.0124 - acc: 0.9995
Epoch 8/15
235/235 [==============================] - 7s 29ms/step - loss: 0.0039 - acc: 0.9997
Epoch 9/15
235/235 [==============================] - 7s 29ms/step - loss: 0.0033 - acc: 0.9997
Epoch 10/15
235/235 [===========

In [ ]:
#四层加dense层(第一轮胜利)
#调整unit 128 最终胜出
#


import random
import tensorflow as tf
seed = 1

import random
import numpy as np
import tensorflow as tf
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


# importing system libraries

from os import walk
from string import punctuation
from random import shuffle
from collections import Counter
from sklearn.model_selection import cross_val_predict
# importing additional libraries

import pandas as pd
import sklearn as sk
import nltk
import pytest
nltk.download('stopwords')
import time
start_time = time.time()



from nltk.test.classify_fixt import setup_module
setup_module()
from os import walk
from string import punctuation
from random import shuffle
from collections import Counter
from sklearn.model_selection import cross_val_predict
# importing additional libraries

import pandas as pd
import sklearn as sk
import nltk
import pytest




from nltk.test.classify_fixt import setup_module
setup_module()

import pandas as pd
import numpy as np
import re
import collections

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import keras
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle





# Read the whole data from the Enron Dataset into a variable allData.

pathwalk = walk(r"/content/enron1/")

allHamData, allSpamData = [], []
for root, dr, file in pathwalk:
    if 'ham' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                allHamData.append(" ".join(ip.readlines()))
                
    elif 'spam' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                allSpamData.append(" ".join(ip.readlines()))
                
                
                

allHamData = list(set(allHamData))
allSpamData = list(set(allSpamData))


hamPlusSpamData = allHamData + allSpamData
labels = ["ham"]*len(allHamData) + ["spam"]*len(allSpamData)

raw_df = pd.DataFrame({"email": hamPlusSpamData, 
                       "label": labels})



raw_df.sample(5)
raw_df['email']

""""""""""""""""""""""""""""""""""""""""""


def preprocess(data):
    # tokenization
    tokens = nltk.word_tokenize(data)
    tokens = [w.lower() for w in tokens if w.isalpha()]

    # finding uncommon words
    cnt = Counter(tokens)
    uncommons = cnt.most_common()[:-int(len(cnt)*0.1):-1]
    
    # listing stopwords from NLTK
    stops = set(nltk.corpus.stopwords.words('english'))

    # removing stop words and uncommon words
    tokens = [w for w in tokens if (w not in stops and w not in uncommons)]

    # lemmatization
    lemmatizer = nltk.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w, pos='a') for w in tokens]

    return tokens

label_encoder = sk.preprocessing.LabelEncoder()
raw_df['label'] = label_encoder.fit_transform(raw_df.label)
import string
from nltk.corpus import stopwords

def process_text(text):
    no_punc = [char for char in text if char not in string.punctuation]
    no_punc = ''.join(no_punc)
    return ' '.join([word for word in no_punc.split() if word.lower() not in stopwords.words('english')])

raw_df['email']=raw_df['email'].apply(process_text)
raw_df['email']

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stemming (text):
    return ''.join([stemmer.stem(word) for word in text])
raw_df['email']=raw_df['email'].apply(stemming)
raw_df.head()


from sklearn.feature_extraction.text import CountVectorizer
vectorizer= CountVectorizer()
message_bow = vectorizer.fit_transform(raw_df['email'])

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(message_bow,raw_df['label'],test_size=0.25)

X = message_bow
y = raw_df['label']




# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(message_bow)

# Transform entire BoW into tf-idf corpus
message_bow_tfidf = tfidf_transformer.transform(message_bow)

X_tfidf = message_bow_tfidf
y = raw_df['label']
X_tfidf_train,X_tfidf_test,y_tfidf_train,y_tfidf_test = train_test_split(message_bow_tfidf,raw_df['label'],test_size=0.25)



texts1 = np.asarray(message_bow)
labels1 = np.asarray(raw_df['label'])



import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

import keras
from keras.models import Sequential
from keras.layers import Dense


from keras.layers import SimpleRNN, Embedding, Dense, LSTM
from keras.models import Sequential

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# number of words used as features
max_features = 500000
# cut off the words after seeing 500 words in each document(email)
maxlen = 500


# we will use 80% of data as training, 20% as validation data
training_samples = int(4994 * .75)
validation_samples = int(4994 - training_samples)
# sanity check
print(len(raw_df['email']) == (training_samples + validation_samples))
print("The number of training {0}, validation {1} ".format(training_samples, validation_samples))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(raw_df['email'])
sequences = tokenizer.texts_to_sequences(raw_df['email'])

word_index = tokenizer.word_index
print("Found {0} unique words: ".format(len(word_index)))

data = pad_sequences(sequences, maxlen=maxlen)

print("data shape: ", data.shape)


#os.environ['PYTHONHASHSEED']=str(SEED)
#random.seed(SEED) 
#np.random.seed(SEED)
#tf.random.set_seed(SEED)
#tf.compat.v1.set_random_seed(123)
# shuffle data
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = np.asarray(labels)
labels = y[indices]


texts_train = data[:training_samples]
y_train = labels[:training_samples]
texts_test = data[training_samples:]
y_test = labels[training_samples:]


from tensorflow.keras.optimizers import RMSprop


max_features =500000
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128))
model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=RMSprop(learning_rate=0.1), loss='binary_crossentropy', metrics=['acc'])
history_ltsm = model.fit(texts_train, y_train, epochs=1, batch_size=10, shuffle=False)

pred = (model.predict(texts_test) > 0.5).astype("int32")

acc = model.evaluate(texts_test, y_test)
proba_ltsm = (model.predict(texts_test) > 0.5).astype("int32")
from sklearn.metrics import confusion_matrix
print("Test loss is {0:.4f} accuracy is {1:.4f}  ".format(acc[0],acc[1]))

from sklearn.metrics import classification_report
print("LSTM score",classification_report(y_test,pred, digits=4))
print(confusion_matrix(pred, y_test))

from pandas.core.common import random_state

from keras.layers.core import Flatten



def create_keras_model():

    max_features =500000
    model2 = Sequential()
    model2.add(Embedding(max_features, 128))
    model2.add(LSTM(128, return_sequences=True))
    model2.add(LSTM(128, return_sequences=True))
    model2.add(LSTM(128, return_sequences=True))
    model2.add(LSTM(128))
    model2.add(Dense(128, activation='relu'))
    model2.add(Dense(1, activation='sigmoid'))
#    rmsprop1 = RMSprop(learning_rate = 0.1)
    model2.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['acc'])



    return model2


from keras.wrappers.scikit_learn import KerasClassifier

LSTMmodel1 = KerasClassifier(
        build_fn=create_keras_model, 
        batch_size=16, epochs=20)


LSTMmodel1._estimator_type = "classifier"






from sklearn.model_selection import cross_val_score
#accuracies_LSTM1 = cross_val_score(estimator=LSTMmodel1, scoring="accuracy", X=data, y=labels, cv=10)
#accuracies_LSTM2 = cross_val_score(estimator=LSTMmodel1, scoring="precision", X=data, y=labels, cv=10)
#print("LSTM corss validation :  %0.4f accuracy with a standard deviation of %0.4f" % (accuracies_LSTM1.mean(), accuracies_LSTM1.std()))
#print("LSTM corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_LSTM2.mean(), accuracies_LSTM2.std()))


accuracies_LSTMmodel1_accuracy = cross_val_score(estimator=LSTMmodel1, scoring="accuracy", 
    X=data, y=labels, cv=10)
accuracies_LSTMmodel1_f1 = cross_val_score(estimator=LSTMmodel1, scoring="f1", 
    X=data, y=labels, cv=10)
accuracies_LSTMmodel1_recall = cross_val_score(estimator=LSTMmodel1, scoring="recall", 
    X=data, y=labels, cv=10)
accuracies_LSTMmodel1_precision = cross_val_score(estimator=LSTMmodel1, scoring="precision", 
    X=data, y=labels, cv=10)

print("unit 128")
print("----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("LSTMmodel1 corss validation :  %0.4f accuracy  with a standard deviation of %0.4f" % (accuracies_LSTMmodel1_accuracy.mean(), accuracies_LSTMmodel1_accuracy.std()))
print("LSTMmodel1 corss validation :  %0.4f f1     with a standard deviation of %0.4f" % (accuracies_LSTMmodel1_f1.mean(), accuracies_LSTMmodel1_f1.std()))
print("LSTMmodel1 corss validation :  %0.4f recall   with a standard deviation of %0.4f" % (accuracies_LSTMmodel1_recall.mean(), accuracies_LSTMmodel1_recall.std()))
print("LSTMmodel1 corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_LSTMmodel1_precision.mean(), accuracies_LSTMmodel1_precision.std()))




